![DSA-Logo.png](.\images\DSA-Logo.png)

# <font color='black'>Projeto:</font> <font color='blue'>Prevendo o Nível de Satisfação dos Clientes do Santander.</font>

## <font color='black'>Data:</font> <font color='blue'>Maio, 2020</font>

## <font color='black'>Cientista de Dados:</font> <font color='blue'>Walter Trevisan</font>

## 02- Get The Data:

Nesta etapa, vamos realizar as seguintes atividades:

1. **Descompactar** e **Carregar** o nosso dataset;

2. Analisar se existem **Registros duplicados** em nosso dataset;

3. Analisar se existem **Missing Values** em nosso dataset;

4. Criar um **Conjunto de Dados de Teste**;

5. Salvar os datasets (**`arquivos csv`**) de **Treino** e **Teste**;

6. Salvar os objetos (**`Data Frames`**) de **Treino** e **Teste**.

## Descrição do Dataset:

>**train.csv**: este é o “dataset” que será utilizado para criarmos o nosso modelo de “Machine Learning”. Este dataset consiste em várias variáveis preditoras (independentes) e uma variável “alvo” (dependente), “**`TARGET`**”, cujo rótulo será substituido para “**`Satisfaction`**”.

## Descrição das Variáveis Preditoras (Independentes):

O nosso dataset possui **`370`** variáveis preditoras anônimas, ou seja, nesta etapa do processo não temos como detalhar o significado de cada variável preditora.

## Descrição da Variável Target (Dependente):

01. **`Satisfaction`**: classe (**"0"** – para **clientes safisfeitos** / **"1"** – para **clientes insafisfeitos**).

## Setup
>### Pacotes e Funções

In [1]:
# As novas versões do Pandas e Matplotlib trazem diversas mensagens de aviso ao desenvolvedor.
# Então, vamos desativar essas mensagens.
import sys # O pacote "sys" permite manipulações com o sistema operacional:
import os  # Operation System (Packages and Functions)
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importando o pacote Pandas:
import pandas as pd
import pickle # Para salvar/carregar objetos.

# Definindo o diretório raiz (Root) onde serão armazenados todas as informações
# (Imagens, Objetos, Dados, Modelos de ML, etc...) do projeto.
# Diretório Raiz (Root) do Projeto:
SANTANDER_ROOT_DIR = "."

# Path: onde ficarão armazenados os "Objetos" (Estrututras de Dados) relacionados ao Projeto:
SANTANDER_OBJ_PATH = os.path.join(SANTANDER_ROOT_DIR, "Objects")
# Criando o diretório, se ele não existir:
os.makedirs(SANTANDER_OBJ_PATH, exist_ok=True)

# Path: onde ficarão armazenados os "datasets" (arquivos "csv") e os "Objetos" (Data Frames) do Projeto:
SANTANDER_DATA_PATH = os.path.join(SANTANDER_ROOT_DIR, "Data")
# Criando o diretório, se ele não existir:
os.makedirs(SANTANDER_DATA_PATH, exist_ok=True)

# Path: onde estão armazenadas as classes e funções que serão utilizadas neste notebook:
SANTANDER_LIB_PATH = os.path.join(SANTANDER_ROOT_DIR, "Library")

# Adicionando o diretório ao 'path' do Sistema, para podermos importar classes e funções que serão
# utilizadas neste notebook:
sys.path.append(SANTANDER_LIB_PATH)

# Importando para este notebook, as classes e funções definidas no módulo "DataScience_Library_v1_0":
import DataScience_Library_v1_0 as dslib

# Importando para este notebook, as classes e funções definidas no módulo "DataScience_Plot_Library_v1_0":
import DataScience_Plot_Library_v1_0 as ptlib

# Importando para este notebook, as classes e funções definidas no módulo "DSA_Project_03_Library":
import DSA_Project_03_Library as pjlib

print("Setup Complete!")

Setup Complete!


In [2]:
# Versões dos pacotes usados neste jupyter notebook:
%reload_ext watermark
%watermark -a "Walter Trevisan" --iversions

pandas 0.25.3
Walter Trevisan


## 01- Descompactar e Carregar o Dataset

In [3]:
# Unzip dataset (train.csv):
pjlib.fetch_santander_dataset()

Files and Folders:
train.csv
train.csv.zip


In [4]:
# Load dataset (train.csv):
santander = pjlib.load_santander_dataset()

Loading file: ..\Datasets\train.csv
The label "TARGET" has been replaced by the label "Satisfaction"
Elapsed time: 3.06 seconds.


## 02- Analisando Informações (Observações) Duplicadas no Dataset:

In [5]:
# Verificando se existem registros/linhas duplicadas no DataFrame "santander":
santander.duplicated().sum()

0

## 03- Analisando *Missing Values* (*`NaN`*) no Dataset:

In [6]:
# Verificando se existem valores ausentes no DataFrame "santander":
santander.isnull().sum().sum()

0

## 04- Criando os datasets: treino (`santander_train`) e teste (`satander_test`)

Embora o nosso dataset não seja pequeno (especialmente em relação ao número de atributos), se utilizarmos um método de amostragem puramente aleatório correremos o risco do resultado apresentar um viés significativo de amostragem. Portanto, utilizaremos o método de *amostragem estratificada*, utilizando a nossa variável target ("Satisfaction"), para termos as mesmas proporções de classes (0 e 1) nos dois conjuntos de dados (treino e teste) gerados.

***amostragem estratificada***: a população é dividida em subgrupos homogêneos, chamados de *estratos*, e o número certo de instâncias de cada estrato é amostrado para garantir que o conjunto de testes seja representativo da população em geral.

Vamos comparar os dois métodos de amostragem: ***aleatório x estratificado***:

Primeiro, vamos criar um conjunto de dados de teste utilizando a função *train_test_split* do *Scikit-Learn*, ou seja, utilizando o método de ***amostragem puramente aleatória***:

In [7]:
from sklearn.model_selection import train_test_split

# Criando os datasets de treino e teste através de amostragem puramente aleatória:
train_set, test_set = train_test_split(santander, test_size=0.2, random_state=42)

Agora, vamos criar o nosso conjunto de dados de teste utilizando o método de ***amostragem estratificada*** com base na categoria da nossa variável target (***Satisfaction***):

In [8]:
from sklearn.model_selection import StratifiedShuffleSplit

# Criando a instância:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Criando os datasets de treino e teste utilizando amostragem estratificada:
for train_index, test_index in split.split(santander, santander["Satisfaction"]):
    strat_train_set = santander.loc[train_index]
    strat_test_set = santander.loc[test_index]

Agora, vamos análisar as proporções da categoria **`Satisfaction`** nos conjuntos de dados completo (*santander*), de treino (*strat_train_set*) e de teste (*strat_train_set*):

In [9]:
# Calculando as proporções de cada categoria no conjunto de dados de teste:
proportions_cat = pd.DataFrame({'Santander Set': santander["Satisfaction"].value_counts() / len(santander),
                                'Train Set': strat_train_set["Satisfaction"].value_counts() / len(strat_train_set),
                                'Teste Set': strat_test_set["Satisfaction"].value_counts() / len(strat_test_set)
                               })

proportions_cat                            

,Santander Set,Train Set,Teste Set
0,0.960431,0.960438,0.960405
1,0.039569,0.039562,0.039595


**Ótimo!!!** As proporções de cada *classe* (0 e 1) são praticamente iguais nos três datasets!

Vamos comparar as proporções da categoria de **`Satisfaction`** no conjunto de dados completo (**santander**), no conjunto de teste gerado com a amostragem estratificada (**strat_test_set**), e em um conjunto de teste gerado a partir da amostragem puramente aleatória (**test_set**):

In [10]:
# Definindo uma função para calcular as proporções de cada categoria da variável target "Satisfaction":
def santander_cat_proportions(data):
    return data["Satisfaction"].value_counts() / len(data)

# Criando um DataFrame para comparação de viés de amostragem estratificada versus amostragem aleatória: 
compare_props = pd.DataFrame({
    "Overall": santander_cat_proportions(santander),
    "Stratified": santander_cat_proportions(strat_test_set),
    "Random": santander_cat_proportions(test_set),
}).sort_index()

# Calculando a taxa de erro da amostragem aleatória:
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100

# Calculando a taxa de erro da amostragem estratificada:
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [11]:
# Visualizando os resultados:
compare_props

,Overall,Stratified,Random,Rand. %error,Strat. %error
0,0.960431,0.960405,0.960076,-0.036980,-0.002739
1,0.039569,0.039595,0.039924,0.897606,0.066489


Como podemos observar nos resultados acima, o conjunto de teste gerado com a utilização da amostragem estratificada (**Stratified**) têm proporções da categoria de *satisfação do cliente* (variável **Satisfaction**) quase idênticas às do conjunto completo de dados (**Overall**), enquanto que o conjunto de testes gerado com amostragem puramente aleatória (**Random**) é bastante distorcido as proporções (as taxas de erro são bem maiores).

Portanto, concluímos a etapa de ***Get The Data*** (Obter os Dados), onde criamos os nosso conjuntos de dados de treino (***strat_train_set***) e de teste (***strat_test_set***):

In [12]:
# Reset no índice do data frame de treinamento:
strat_train_set.reset_index(drop=True, inplace=True)

# Visualizando as primeiras instâncias do dataset de treino (strat_train_set):
strat_train_set.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,Satisfaction
0,111813,2,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
1,31053,2,79,105.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,68480.010000,0
2,136647,2,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,57517.740000,0
3,100343,2,29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60060.930000,0
4,9720,2,25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84028.740000,0


In [13]:
# Reset no índice do data frame de teste:
strat_test_set.reset_index(drop=True, inplace=True)

# Visualizando as primeiras instâncias do dataset de teste (strat_test_set):
strat_test_set.head()

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,...,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,Satisfaction
0,125987,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979016,0
1,54150,2,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72380.010000,0
2,11883,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,121334.280000,0
3,9733,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,114830.010000,0
4,12745,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,82863.900000,0


In [14]:
print(len(strat_train_set), "train +", len(strat_test_set), "test")

60816 train + 15204 test


## 05- Salvando os datasets: treino (`santander_train.csv`) e teste (`santander_test.csv`):

In [15]:
# Save "strat_train_set" to "santander_train.csv" file:
train_file = os.path.join(SANTANDER_DATA_PATH, "santander_train.csv")
strat_train_set.to_csv(path_or_buf=train_file,index=False)

# Save "strat_test_set" to "santander_test.csv" file:
test_file = os.path.join(SANTANDER_DATA_PATH, "santander_test.csv")
strat_test_set.to_csv(path_or_buf=test_file,index=False)

## 06- Salvando os DataFrames: treino (`santander_train`) e teste (`santander_test`):

Agora, vamos salvar os `Data Frames` importantes (`Train` e `Test`), que serão utilizados nas próximas etapas, a saber:

1. **`santander_train`:** data frame de treino criado por **amostragem estratificada** a partir do data frame `santander`;

2. **`santander_test`:** data frame de teste criado por **amostragem estratificada** a partir do data frame `santander`.

In [16]:
# Salvando o data frame "strat_train_set":
dslib.pickle_object_save (path=SANTANDER_DATA_PATH, file="santander_train.pickle", object_name=strat_train_set,
                          msg="The 'strat_train_set' (data frame) has been successfully saved!")

# Salvando o data frame "strat_train_set":
dslib.pickle_object_save (path=SANTANDER_DATA_PATH, file="santander_test.pickle", object_name=strat_test_set,
                          msg="The 'strat_test_set' (data frame) has been successfully saved!")

The 'strat_train_set' (data frame) has been successfully saved!
The 'strat_test_set' (data frame) has been successfully saved!


## <font color='black'>FIM</font>